In [7]:
from bs4 import BeautifulSoup
import urllib.request
import requests
from pprint import pprint
import pandas as pd
import json
import copy
from datetime import datetime, timezone
from google.cloud import storage
import gcsfs
import re
from typing import List

In [8]:
storage_client = storage.Client(project="ytone-430507")
today = datetime.now()
table_name = "douyin_video"
api_name = "get_author_show_items_v2"

In [9]:
video_data = []
video_columns = ["video_id", "item_id", "title", "image_url", "comment", "like", "share", "play", "create_time", "duration", "influencer_id"]

In [10]:
processing_blobs = [
{
    "blob": blob,
    "date": datetime.strptime(blob.name.split('/')[1], "%Y-%m-%d"),
    "batch_number": int(blob.name.split('/')[-1].replace(".json", "").split("_")[-1]),
} for blob in storage_client.list_blobs("0_raw_data",prefix="2_xingtu/") if api_name in blob.name]
processing_blobs

[{'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-11/get_author_show_items_v2_0911_0.json, 1726037775725233>,
  'date': datetime.datetime(2024, 9, 11, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-17/get_author_show_items_v2_0917_0.json, 1726565277410211>,
  'date': datetime.datetime(2024, 9, 17, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-18/get_author_show_items_v2_0918_0.json, 1726633856126752>,
  'date': datetime.datetime(2024, 9, 18, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-19/get_author_show_items_v2_0919_0.json, 1726719807869764>,
  'date': datetime.datetime(2024, 9, 19, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-10-09/get_author_show_items_v2_241009_0.json, 1728447750932489>,
  'date': datetime.datetime(2024, 10, 9, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-10-10/get_author_show_items_v2_1010_0.json, 1728549328732271>,
  'date': da

In [11]:
processed_blobs = [{
    "blob": blob,
    "date": datetime.strptime(blob.name.split('/')[2], "%Y-%m-%d"),
    "batch_number": int(blob.name.split('/')[-1].replace(".parquet", "").split("_")[-1]),
} for blob in storage_client.list_blobs("4_data_warehouse",prefix="1_xingtu/") if table_name == blob.name.split("/")[1] and len(blob.name.split("/")) == 4]
processed_blobs

[{'blob': <Blob: 4_data_warehouse, 1_xingtu/douyin_video/2024-09-08/douyin_video_240918_0.parquet, 1726637067624622>,
  'date': datetime.datetime(2024, 9, 8, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 4_data_warehouse, 1_xingtu/douyin_video/2024-09-08/douyin_video_240918_1.parquet, 1726637228209113>,
  'date': datetime.datetime(2024, 9, 8, 0, 0),
  'batch_number': 1},
 {'blob': <Blob: 4_data_warehouse, 1_xingtu/douyin_video/2024-09-11/douyin_video_240918_0.parquet, 1726641889438713>,
  'date': datetime.datetime(2024, 9, 11, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 4_data_warehouse, 1_xingtu/douyin_video/2024-09-17/douyin_video_240918_0.parquet, 1726642029290605>,
  'date': datetime.datetime(2024, 9, 17, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 4_data_warehouse, 1_xingtu/douyin_video/2024-09-18/douyin_video_240918_0.parquet, 1726652981717438>,
  'date': datetime.datetime(2024, 9, 18, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 4_data_warehouse, 1_xingtu/douyin_video/2024-

In [12]:
to_process = []
for processing_blob in processing_blobs:
    if processing_blob["date"] >= datetime(2024, 9, 13):
        if processing_blob["batch_number"] not in [processed_blob["batch_number"] for processed_blob in processed_blobs if processing_blob["date"] == processed_blob["date"]]:
            to_process.append(processing_blob)
pprint(to_process)
print(len(to_process))

[{'batch_number': 0,
  'blob': <Blob: 0_raw_data, 2_xingtu/2024-10-11/get_author_show_items_v2_1011_0.json, 1728632787878432>,
  'date': datetime.datetime(2024, 10, 11, 0, 0)}]
1


In [13]:
def get_author_show_items_v2_data(raw_data):
    result = []
    for data in raw_data:
        get_author_show_items_v2 = data.get("get_author_show_items_v2")
        profile_id = data["profile_id"]
        if get_author_show_items_v2:
            latest_item_info = get_author_show_items_v2.get("latest_item_info")
            latest_star_item_info = get_author_show_items_v2.get("latest_star_item_info")
            items = latest_item_info + latest_star_item_info
            if items:
                for item in items:
                    record = {}
                    record["comment"] = item["comment"]
                    record["like"] = item["like"]
                    record["share"] = item["share"]
                    record["play"] = item["play"]
                    record["video_id"] = item["video_id"]
                    record["item_id"] = item["item_id"]
                    record["title"] = item["title"]
                    record["image_url"] = item["item_cover"]
                    record["create_time"] = str(datetime.fromtimestamp(int(item["create_time"])).replace(tzinfo=timezone.utc))
                    record["duration"] = item["duration"]
                    record["influencer_id"] = profile_id
                    result.append(record)
    return result


In [14]:
for item in to_process:
    raw_data = json.loads(item["blob"].download_as_string())
    video_data = get_author_show_items_v2_data(raw_data)
    df = pd.DataFrame(video_data, columns=video_columns)
    df.to_parquet("gs://4_data_warehouse/1_xingtu/douyin_video/" + str(item["date"].date()) + "/douyin_video_" + today.strftime("%y%m%d") + "_" + str(item["batch_number"]) + ".parquet")